# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [28]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import cartopy
import geoviews
import pyproj

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,11.57,92,100,0.45,NZ,1738939659
1,1,sturgis,41.7992,-85.4192,-4.40,71,20,4.12,US,1738939660
2,2,edinburgh of the seven seas,-37.0676,-12.3116,15.18,67,34,6.86,SH,1738939661
3,3,anadyr,64.7500,177.4833,-23.72,76,40,2.00,RU,1738939662
4,4,iskateley,67.6803,53.1512,-0.76,93,100,10.70,RU,1738939663


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humidity = city_data_df["Humidity"].astype(int)

map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,11.57,92,100,0.45,NZ,1738939659
1,1,sturgis,41.7992,-85.4192,-4.40,71,20,4.12,US,1738939660
2,2,edinburgh of the seven seas,-37.0676,-12.3116,15.18,67,34,6.86,SH,1738939661
3,3,anadyr,64.7500,177.4833,-23.72,76,40,2.00,RU,1738939662
4,4,iskateley,67.6803,53.1512,-0.76,93,100,10.70,RU,1738939663
...,...,...,...,...,...,...,...,...,...,...
566,566,zaragoza,41.6561,-0.8773,10.57,65,75,1.54,ES,1738940332
567,567,manokwari,-0.8667,134.0833,27.05,80,84,3.74,ID,1738940333
568,568,yarmouth,41.7057,-70.2286,5.05,87,100,5.81,US,1738940334
569,569,bodaybo,57.8506,114.1933,-38.51,100,41,0.78,RU,1738940335


### Step 3: Create a new DataFrame called `hotel_df`.

In [20]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
city = city_data_df["City"]
country = city_data_df["Country"]
lat = city_data_df["Lat"]
lng = city_data_df["Lng"]
humidity = city_data_df["Humidity"]

hotel_df = city_data_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,waitangi,-43.9535,-176.5597,11.57,92,100,0.45,NZ,1738939659,
1,1,sturgis,41.7992,-85.4192,-4.40,71,20,4.12,US,1738939660,
2,2,edinburgh of the seven seas,-37.0676,-12.3116,15.18,67,34,6.86,SH,1738939661,
3,3,anadyr,64.7500,177.4833,-23.72,76,40,2.00,RU,1738939662,
4,4,iskateley,67.6803,53.1512,-0.76,93,100,10.70,RU,1738939663,
...,...,...,...,...,...,...,...,...,...,...,...
566,566,zaragoza,41.6561,-0.8773,10.57,65,75,1.54,ES,1738940332,
567,567,manokwari,-0.8667,134.0833,27.05,80,84,3.74,ID,1738940333,
568,568,yarmouth,41.7057,-70.2286,5.05,87,100,5.81,US,1738940334,
569,569,bodaybo,57.8506,114.1933,-38.51,100,41,0.78,RU,1738940335,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [24]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
waitangi - nearest hotel: Hotel Chathams
sturgis - nearest hotel: Travelodge Sturgis
edinburgh of the seven seas - nearest hotel: No hotel found
anadyr - nearest hotel: Гостевой дом
iskateley - nearest hotel: Авантаж
blackmans bay - nearest hotel: Villa Howden
mount gambier - nearest hotel: The Old Mount Gambier Gaol
el hadjira - nearest hotel: No hotel found
port mathurin - nearest hotel: Escale Vacances
aykhal - nearest hotel: Уют
port-aux-francais - nearest hotel: Keravel
broome - nearest hotel: No hotel found
geraldton - nearest hotel: Ocean Centre Hotel
nikolayevsk-on-amure - nearest hotel: Север
fort st. john - nearest hotel: Fort St. John Motor Inn
papatowai - nearest hotel: No hotel found
bikenibeu village - nearest hotel: Otintaai Hotel
adamstown - nearest hotel: No hotel found
i-n-salah - nearest hotel: باجودة
remire-montjoly - nearest hotel: Complexe Belova
port elizabeth - nearest hotel: Waterford Hotel
qaqortoq - nearest hotel: Hotel Qaqortoq
albany -

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,waitangi,-43.9535,-176.5597,11.57,92,100,0.45,NZ,1738939659,Hotel Chathams
1,1,sturgis,41.7992,-85.4192,-4.40,71,20,4.12,US,1738939660,Travelodge Sturgis
2,2,edinburgh of the seven seas,-37.0676,-12.3116,15.18,67,34,6.86,SH,1738939661,No hotel found
3,3,anadyr,64.7500,177.4833,-23.72,76,40,2.00,RU,1738939662,Гостевой дом
4,4,iskateley,67.6803,53.1512,-0.76,93,100,10.70,RU,1738939663,Авантаж
...,...,...,...,...,...,...,...,...,...,...,...
566,566,zaragoza,41.6561,-0.8773,10.57,65,75,1.54,ES,1738940332,Hotel Pilar Plaza
567,567,manokwari,-0.8667,134.0833,27.05,80,84,3.74,ID,1738940333,Hotel Arfak
568,568,yarmouth,41.7057,-70.2286,5.05,87,100,5.81,US,1738940334,The Inn At Cape Cod
569,569,bodaybo,57.8506,114.1933,-38.51,100,41,0.78,RU,1738940335,НОЙ


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [30]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)